## OPEN API AND LLAMA
### This project makes calls to 2 LLMs, Open API and Llama to generate, website summaries.

In [ ]:
# First cell: Set up environment and imports
import sys
import os
from pathlib import Path


# Add the src directory to sys.path instead of project_root
current_path = Path(os.path.abspath(''))
src_path = current_path if current_path.name == 'src' else current_path / 'src'
sys.path.insert(0, str(src_path))

# Import without the src prefix
from llm.llm_factory import LLMFactory
from main_summarize import summarize_url
from helper.display_utils import display_summary_markdown
from llm.helper.prompt_utils import PromptManager
from IPython.display import Markdown, display

# Helper function to display section headers
def display_section(title):
    display(Markdown(f"## {title}"))
    print("-" * 80)

# Helper function to display results
def display_result(provider, url, summary):
    print(f"\n🌐 URL: {url}")
    print(f"🤖 Provider: {provider}\n")
    display_summary_markdown(summary)
    print("-" * 80)

# Make sure we have all required dependencies
import importlib
import subprocess
import sys

def ensure_dependencies():
    required_packages = {
        'requests': 'requests',
        'beautifulsoup4': 'bs4',
        'selenium': 'selenium',
        'openai': 'openai',
        'webdriver-manager': 'webdriver_manager',
    }

    missing_packages = []
    for package_name, import_name in required_packages.items():
        try:
            importlib.import_module(import_name)
            print(f"✓ {package_name} is already installed")
        except ImportError:
            missing_packages.append(package_name)
            print(f"✗ {package_name} needs to be installed")

    if missing_packages:
        print("\nInstalling missing packages...")
        for package in missing_packages:
            print(f"Installing {package}...")
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "install", package])
                print(f"Successfully installed {package}")
            except subprocess.CalledProcessError:
                print(f"Failed to install {package}")

    all_installed = True
    for package_name, import_name in required_packages.items():
        try:
            importlib.import_module(import_name)
        except ImportError:
            all_installed = False
            print(f"⚠️ {package_name} installation failed")

    return all_installed

ensure_dependencies()

### Run Example 1: Basic Website Summary using OpenAI and Llama

In [ ]:
# %% [markdown]
# ## Example 1: Basic Website Summary using OpenAI and Llama

display_section("Example 1: Basic Website Summary")

# Default system and user prompts (showing the literal values)
DEFAULT_SYSTEM_PROMPT = ("You are an assistant that analyzes the contents of a website "
                         "and provides a short summary, ignoring text that might be navigation related. "
                         "Respond in markdown.")

DEFAULT_USER_PROMPT_TEMPLATE = """
You are looking at a website titled {title}
The contents of this website is as follows; 
please provide a short summary of this website in markdown.
If it includes news or announcements, then summarize these too.

{text}
"""

prompt_manager = PromptManager(DEFAULT_SYSTEM_PROMPT, DEFAULT_USER_PROMPT_TEMPLATE)

# Define the URL to analyze
# sample_url = "https://example.com"
sample_url = "https://andresmendoza.dev"

# OpenAI commented; it's expensive
# ===== OpenAI Example (can be commented out to avoid API charges) =====
try:
    # Initialize and validate OpenAI client
    openai_client = LLMFactory.create_client("openai")
    is_valid, message = openai_client.validate_credentials()
    
    if is_valid:
        print("✅ OpenAI credentials validated successfully")
        
        # Generate summary
        openai_summary = summarize_url(
            openai_client, 
            sample_url, 
            prompt_manager=prompt_manager,
            use_selenium=False  # Default, explicitly shown for clarity
        )
        
        # Display results
        display_result("OpenAI", sample_url, openai_summary)
    else:
        print(f"❌ OpenAI validation failed: {message}")
except Exception as e:
    print(f"❌ Error with OpenAI: {str(e)}")

# ===== Llama Example =====
try:
    # Initialize and validate Llama client
    llama_client = LLMFactory.create_client("llama")
    is_valid, message = llama_client.validate_credentials()
    
    if is_valid:
        print("✅ Llama credentials validated successfully")
        
        # Generate summary
        llama_summary = summarize_url(
            llama_client, 
            sample_url, 
            prompt_manager=prompt_manager,
            use_selenium=False  # Default, explicitly shown for clarity
        )
        
        # Display results
        display_result("Llama", sample_url, llama_summary)
    else:
        print(f"❌ Llama validation failed: {message}")
except Exception as e:
    print(f"❌ Error with Llama: {str(e)}")


### Run Example 2: Technical Documentation Summary using Custom Prompts

In [ ]:
# %% [markdown]
# ## Example 2: Technical Documentation Summary using Custom Prompts

display_section("Example 2: Technical Documentation Summary using Custom Prompts")

# Create custom prompt manager for technical documentation
TECH_SYSTEM_PROMPT = """You are a technical documentation specialist. 
Analyze the provided website content and generate a concise technical summary.
Focus on technical details, features, and specifications while ignoring navigation elements.
Respond using well-structured markdown."""

TECH_USER_PROMPT = """
You're analyzing a technical website titled: {title}

Please provide:
1. A brief overview (2-3 sentences)
2. Key technical features or specifications (3-5 bullet points)
3. Target audience or use cases
4. Any technical requirements mentioned

Website content:
{text}
"""

tech_prompt_manager = PromptManager(TECH_SYSTEM_PROMPT, TECH_USER_PROMPT)

# From now on, we'll use Llama (to avoid OpenAI costs)
personal_website_url = "https://andresmendoza.dev"

try:
    # We've already validated Llama client above, so we can reuse it
    if 'llama_client' in locals() and is_valid:
        print("🔍 Analyzing personal website with technical documentation prompts")
        
        # Generate technical summary with custom prompts
        tech_summary = summarize_url(
            llama_client,
            personal_website_url,
            prompt_manager=tech_prompt_manager,
            use_selenium=False  # Most personal websites don't need Selenium
        )
        
        # Display results
        display_result("Llama (Technical Analysis)", personal_website_url, tech_summary)
    else:
        print("❌ Llama client not available or invalid")
except Exception as e:
    print(f"❌ Error analyzing personal website: {str(e)}")



### Run Example 3: Analyzing JavaScript-Heavy SPA with Selenium

In [ ]:
# Examples 3 and 4 take too long; commented
# %% [markdown]
# ## Example 3: Analyzing JavaScript-Heavy SPA with Selenium

display_section("Example 3: Analyzing JavaScript-Heavy SPA with Selenium")

# Create a prompt manager for SPA analysis
SPA_SYSTEM_PROMPT = """You are a frontend analysis specialist. 
Analyze this Single Page Application (SPA) website and provide insights about its structure,
features, and user experience. Focus on identifying framework-specific elements and patterns.
Respond in well-formatted markdown."""

SPA_USER_PROMPT = """
You're analyzing a Single Page Application titled: {title}

Based on the extracted content, please provide:
1. A brief overview of the site's purpose and functionality
2. Identification of likely frontend framework used (Vue, React, Angular, etc.)
3. Notable UI/UX features and interaction patterns
4. Performance observations (if any indicators are present)

Extracted SPA content:
{text}
"""

spa_prompt_manager = PromptManager(SPA_SYSTEM_PROMPT, SPA_USER_PROMPT)

# Choose a JavaScript framework site to analyze
framework_urls = {
    "vue": "https://vuejs.org/",
    "angular": "https://angular.io/",
    "react": "https://reactjs.org/"
}

# Let's analyze the Vue.js website as an example
spa_url = framework_urls["vue"]

try:
    if 'llama_client' in locals() and is_valid:
        print("🔍 Analyzing JavaScript framework website (requires Selenium)")
        
        # Generate summary for SPA site with Selenium enabled
        spa_summary = summarize_url(
            llama_client,
            spa_url,
            prompt_manager=spa_prompt_manager,
            use_selenium=True  # Enable Selenium for JavaScript-heavy sites
        )
        
        # Display results
        display_result("Llama (SPA Analysis with Selenium)", spa_url, spa_summary)
    else:
        print("❌ Llama client not available or invalid")
except Exception as e:
    print(f"❌ Error analyzing SPA website: {str(e)}")

### Run Example 4: Comparative Analysis of Multiple Websites

In [ ]:
# %% [markdown]
# ## Example 4: Comparative Analysis of Multiple Websites

display_section("Example 4: Comparative Analysis of Multiple Websites")

# Create a prompt manager for comparative analysis
COMPARATIVE_SYSTEM_PROMPT = """You are a digital content analyst specializing in comparative analysis.
Your task is to analyze website content and place it in context with similar sites in its category.
Focus on identifying unique selling points, target audience, and distinguishing features.
Respond in well-formatted markdown."""

COMPARATIVE_USER_PROMPT = """
You're analyzing a website titled: {title}

Based on the content, please provide:
1. A concise summary of the site's purpose and content
2. How this site appears to differentiate itself from competitors
3. Target audience analysis
4. Content quality assessment (professionalism, clarity, comprehensiveness)

Website content:
{text}
"""

comparative_prompt_manager = PromptManager(COMPARATIVE_SYSTEM_PROMPT, COMPARATIVE_USER_PROMPT)

# Let's analyze a different JavaScript framework for comparison
comparison_url = framework_urls["angular"]

try:
    if 'llama_client' in locals() and is_valid:
        print("🔍 Performing comparative analysis of another framework website")
        
        # Generate comparative summary
        comparative_summary = summarize_url(
            llama_client,
            comparison_url,
            prompt_manager=comparative_prompt_manager,
            use_selenium=True  # Enable Selenium for JavaScript-heavy sites
        )
        
        # Display results
        display_result("Llama (Comparative Analysis)", comparison_url, comparative_summary)
    else:
        print("❌ Llama client not available or invalid")
except Exception as e:
    print(f"❌ Error performing comparative analysis: {str(e)}")

print("*" * 80)
print("END")
print("*" * 80)

# %% [markdown]
# ## Summary of Examples
# 
# This notebook demonstrated:
# 
# 1. Basic website summarization with both OpenAI and Llama
# 2. Technical documentation analysis with custom prompts
# 3. SPA website analysis using Selenium for JavaScript-heavy sites
# 4. Comparative website analysis
# 
# Each section can be run independently or commented out as needed.